In [27]:
from scipy.stats import multivariate_normal
import numpy as np
import numpy.matlib
import math
%run Rich_Interactions_in_MR\performance_utils.py

In [28]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

# Sequence Decoding for Text Entry

Let us assume we have a simple touchscreen keyboard which can generate $k$ distinct letters $L=\left\{l_1, l_2, \ldots, l_k\right\}$.

We view our signal as a discrete <em>observation sequence</em>, which, for ease of notation, we will denote $O$. When a user is typing on the touchscreen keyboard we are provided with a touch point coordinate $(x,y)$, where $x$ and $y$ are the horizontal and vertical components of the touch point respectively. A set of $n$ touch points then form $n$ observations:

\begin{equation}
O = \left\{o^{(1)}, o^{(2)}, \ldots, o^{(n)}\right\}
\end{equation}

Since subscripts are often used to denote specific members of a set, we use a superscript within parentheses to denote the index of an individual observation in an sequence (indexing starts at 1). This means that for instance the sequence $l_1, l_2$ means two letters selected from the set of letters that can be generated by the keyboard (which, since we have defined the keyboard above to generate distinct letters, means $l_1 \neq l_2$). In contrast, $l_1^{(1)},l_1^{(2)}$ means we have generated the _same_ letter $l_1$ twice. Similarly, $l_1^{(1)},l_2^{(2)}$ means we have generated the letter sequence $l_1, l_2$.

We are now given a set of observations $O = \left\{o^{(1)}, o^{(2)}, \ldots, o^{(n)}\right\}$ and we wish to find a _hypothesis_ for $O$, which is a corresponding letter sequence ${L} = \left\{l^{(1)}, l^{(2)}, \ldots, l^{(m)}\right\}$. Note that the cardinality of $O$ and $L$ are not necessarily identical, that is, it is not neccesarily true that $n = m$ ($|O|=|L|$). There are obviously many possible letter sequences that can match the observation sequence. However, we are only interested in the most _probable_ letter sequence $\hat{L}$, and this can be computed using the expression we derived earlier:

\begin{equation}
\hat{L}=\underset{L}{\arg\max}\left[P(O|L)P(L)\right]
\end{equation}

$P(L)$ is the prior. What would be a suitable model for the prior probability of a letter sequence? This particular problem is known as _statistical language modelling_ and it is a research area that has received considerable attention. A very simple model is the unigram probability model, which in this case simply assigns a probability to each individual letter $l_i$. The probabilities assigned over the letters $l_1, l_2, \ldots, l_k$ (remember that our keyboard could generate $k$ distinct letters) then must sum to one and this is our prior probability distribution.

$P(O|L)$ is the likelihood of the observation sequence given a particular letter sequence hypothesis. For a touchscreen keyboard this means that we need a model that assigns a probability to an individual $(x,y)$ touch point given a particular letter key hypothesis, that is $P(o_i|l_i)$. A simple way to achieve this is to centre a two-dimensional Gaussian distribution at each key centre and assume each 2D Gaussian has equal variance and that there is no correlation between the horizontal and vertical axes (that is, the covariance matrix is diagonal). 

<img src="Rich_Interactions_in_MR/imgs/key_with_2d_gaussian.svg">

To compute our likelihood distribution we need to compute the probability of observing a specific touch point for all possible $k$ letters that can be generated by the keyboard. For example, if our keyboard has two keys, say $A$ and $B$ generating the letters $a$ and $b$ respectively, then our likelihood distribution would need to be computed by computing the probability of the touch point for key $A$ and key $B$. If the touch point is centred at key $A$ the probability will be maximised for $A$ and smaller for $B$ (we assume keys do not overlap spatially).

<img src="Rich_Interactions_in_MR/imgs/ab_keys_with_2d_gaussian.svg">

<p>
The search for the most probable letter sequence given an observation sequence can be modelled as a (discrete-time) <em>Markov chain</em>. A Markov chain assumes a future state is conditional on the previously known state and independent of any other previous or future states.
</p>

<h2>Statistical Decoding using Token Passing</h2>

<p>
We will now explain how to statistically decode the observation sequence $O$ into the most probable letter sequence $L$. There are several algorithms for learning a hidden Markov model and performing inference. For our particular decoding problem a flexible and efficient method is known as <em>token passing</em>. A particular advantage is that it can efficiently search more complex models and it can be easily parallelised.
</p>

<h3>Token</h3>

<p>
Fundamental to token passing is the notion of a <em>token</em>. A token is a data structure that at a minimum contains the following information: 1) the hypothesis generated so far; and 2) the accumulated probability of the hypothesis. For a touchscreen keyboard the hypothesis generated so far is a letter sequence (a string). For bookkeeping reasons we will also let each token store which observation index it belongs to.
</p>

In [9]:
class Token:
    def __init__(self, hypo, acc_prob, obs_index):
        self.hypo = hypo
        self.acc_prob = acc_prob
        self.obs_index = obs_index

<h3>Observation</h3>

<p>
An observation is simply an $(x,y)$ touch point.
</p>

In [10]:
class Observation:
    def __init__(self, x, y):
        self.x = x
        self.y = y

<h3>Observation Sequence</h3>

<p>
The observation sequence $O = \left\{o^{(1)}, o^{(2)}, \ldots, o^{(n)}\right\}$ is simply a series of observations. They could be kept in a list but it is often convenient to represent them as a separate class when we later improve the model's performance.
</p>

In [11]:
class ObservationSequence:
    def __init__(self, seq):
        self.seq = seq

<h3>Key</h3>

<p>
A key describes the geometry of the keyboard. In this case our likelihood model is very simple. All we need is information on where to centre the 2D Gaussian, which means we need the centre coordinate of the key, and the letter generated by the key. Below we have generated a simple keyboard layout with four keys: A, B, C and D.
</p>

<img src="Rich_Interactions_in_MR/imgs/key_layout.svg">

In [12]:
class Key:
    def __init__(self, centre_x, centre_y, letter):
        self.centre_x = centre_x
        self.centre_y = centre_y
        self.letter = letter
                
a_key = Key(0.0,0.0,'a')
b_key = Key(0.0,1.0,'b')
c_key = Key(1.0,0.0,'c')
d_key = Key(1.0,1.0,'d')
keys = [a_key, b_key, c_key, d_key]

### Prior

The prior is in this instance the simple unigram probability of the letter. It can be implemented using a simple hash table. To make the demonstration simple we will assume a unigram letter model and all of the letters will be equally probable.

In [13]:
lm = {'a':0.25,'b':0.25,'c':0.25,'d':0.25}
def get_prior(context, lm):
    c = context[-1]
    return lm[c]

### Likelihood

The likelihood is a 2D Gaussian probability density function for the key corresponding to the letter under consideration, evaluated at the coordinate specified by the observation. To make the demonstration simple we will assume a symmetrical 2D Gaussian with unit variance.

In [14]:
def get_likelihood(x, y, key):
    key_x = key.centre_x
    key_y = key.centre_y
    p = multivariate_normal.pdf([x,y], mean=[key_x,key_y], cov=[[1,0],[0,1]]);
    return p

## Substitution-Only Decoder

The first decoder we will look at is a _substitution-only_ decoder. It takes as input:

* A set of $k$ symbols that can be recognised.
* A set of keys (a centre coordinate and a letter, the letter must correspond to a symbol in the symbol set).
* A language model (in this case a simple mapping of a single letter to a probability).
* An observation sequence.

Our decoder will receive a series of touch points, that is $O = \left\{o^{(1)}, o^{(2)}, \ldots, o^{(n)}\right\}$.

To be able to decode such a model we start with a single initial token. The process now works as follows. We take any token at observation index $i \in [0,n)$ and we propagate $k$ tokens to observation index $i+1$. We do this until we are at the last observation and can no longer propagate any tokens. These final tokens represent all hypotheses for the observation sequence. The token with the most probable hypothesis is the most likely letter sequence corresponding to the observation sequence.

The model of the substitution-only decoder is shown below. An initial start state leads to a state each for every observation. States are linked by edges. Each edge links two states and generates a symbol. A series of edges generate a _path_. As is evident in the figure, several paths are possible from the start state to the end state. The subsitution-only decoder below will search _all_ such possible paths and substitute an observation for a symbol every time a particular edge is traversed when transitioning from one state to another.

<img src="Rich_Interactions_in_MR/imgs/simple_chain.svg">

The code below provides several possible observation sequences. The default observation sequence is empty and generates the expected result that the decoder predicts an empty hypothesis (no letter generated) given an empty observation sequence. By commenting and uncommenting the observation sequences it is possible to explore a few observation sequences:

* No observations. This generates a single-state search space.
* A noisy single touch in the vicinity of the A key. This generates a two-state search space.
* Two noisy touches, both in the vicinity of the A key. This generates a three-state search space.
* Two noisy touches near the B key followed by the A key. This generates a three-state search space.
* Three noisy touches near the B key followed by the A key, followed by the D key. This generates a four-state search space (illustrated in the figure above).

<div class="alert alert-info">
<strong>GROUP DISCUSSION:</strong> 
<ul>
    <li>
        How does the number of tokens relate to the number of observations?
    </li>
    <li>
        Why is the letter sequence probability the same for 'b' and 'c' in the sequence with a single observation?
    </li>
</ul>
</div>

In [15]:
symbols = ['a','b','c','d']
#obs_seq = ObservationSequence([]) #No observations
obs_seq = ObservationSequence([Observation(0.1,0.1)]) #a + noise
#obs_seq = ObservationSequence([Observation(0.1,0.1),Observation(0.15,-0.1)]) #aa + noise
#obs_seq = ObservationSequence([Observation(0.1,1.1),Observation(0.15,-0.1)]) #ba + noise
#obs_seq = ObservationSequence([Observation(0.1,1.1),Observation(0.15,-0.1),Observation(1.05,0.95)]) #bad + noise
completed_tokens = []

def get_key_for_symbol(symbol):
    for key in keys:
        if key.letter == symbol:
            return key
    return None

# Convenience function for nicely printing token passing results
def print_sorted_tokens(sorted_tokens):
    results = sorted_tokens
    results = list(zip(range(1,len(results) + 1), results))
    shortened_id = 15
    shortened = False
    if (len(results) > 32):
        results = results[:shortened_id] + results[-shortened_id:]
        shortened = True
    print("letter sequence")
    print("nth best, hypothesis, probability")
    for i, result in results:
        if (result.acc_prob < 1e-7):
            print("%d, %s, %.3g" % (i, result.hypo, result.acc_prob))
        else:
            print("%d, %s, %.7f" % (i, result.hypo, result.acc_prob))
        if (i == shortened_id and shortened):
            print("...")

def propagate(token, obs_seq, completed_tokens):
    # Which observation is the token in?
    ix = token.obs_index
    # Are there more observations?
    next_ix = ix + 1
    if next_ix == len(obs_seq.seq):
        # No more observations, add token to the list of completed tokens
        completed_tokens.append(token)
    else:
        # There is another observation, propagage a token per symbol to the next observation
        next_obs = obs_seq.seq[next_ix]
        for symbol in symbols:
            # Find the key that generates the symbol
            key = get_key_for_symbol(symbol)
            # Calculate the prior probability of our symbol given previous symbols we have generated
            prior = get_prior(token.hypo+symbol, lm)
            # Calculate the likelihood of the observation (touch coordinate) given our current symbol hypothesis
            likelihood = get_likelihood(next_obs.x, next_obs.y, key)
            # Calculate an accumulated probability for our posterior probability
            acc_prob = token.acc_prob * prior * likelihood
            # Create a new token for the next observation index that contains the accumulated hypothesis and probability
            new_token = Token(token.hypo+symbol, acc_prob, next_ix)
            # Propagate our token
            propagate(new_token, obs_seq, completed_tokens)

# To start the process we need to create an initial token which we can then propagate to the first observation index
seed_token = Token("", 1.0, -1)
# Propagating the first token starts the search process
propagate(seed_token, obs_seq, completed_tokens)
# Sort the list of completed tokens according to the probabilities of their hypotheses
results = sorted(completed_tokens, key=lambda token: token.acc_prob, reverse=True)

# Print out sequence hypotheses and their probabilities
print_sorted_tokens(results)

letter sequence
nth best, hypothesis, probability
1, a, 0.0393928
2, b, 0.0264058
3, c, 0.0264058
4, d, 0.0177003


<p>
<div class="alert alert-block alert-success"> 
<h3>Exercise 1</h3> 
<br>
Write a test to independently compute the probabilities of the length 1 and 2 observation sequences examined above by completing the followings stubs.  Ensure you understand what the recursive token propagation method is achieving.
</div>
</p>

<div class="alert alert-block alert-success"> 
<strong>Task 1:</strong> Complete the following stub code to ensure the test passes for the single observation case
</div>

In [16]:
def TestSingleObservation(observation_sequence, sequence_probabilities, tolerance):
    # This flag will stay True if your test passes
    test_passes = True

    # Pull out the single observation
    obs1 = observation_sequence.seq[0]
    
    token_count = 0
    
    # Loop through possible symbols (letter keys)
    for symbol in symbols:
        # Find the key that generates the symbol
        key = get_key_for_symbol(symbol)

        # TODO compute the prior, likelihood and accumulated probability after the single observation
        # ...

        # Compare accumulated probability to that reported from our recursive token passing method
        if not (abs(obs_seq1_prob[token_count] - acc_prob) < tol):
            test_passes = False
        token_count += 1
        
    return test_passes

Run the test to ensure it passes (no code changes required)

In [17]:
# Do not change the tolerance
tol = 0.000001

# A sequence with a single Observation
obs_seq1 = ObservationSequence([Observation(0.1,0.1)]) #a + noise
# The probabilities associated with the four possible token hypotheses, i.e. 'a','b','c','d'
obs_seq1_prob = [0.039393,0.026406,0.026406,0.017700]

if (TestSingleObservation(obs_seq1,obs_seq1_prob,tol)):
    print("PASSED: well done!")
else:
    print("FAILED: try again!")

NameError: name 'acc_prob' is not defined

<div class="alert alert-block alert-success"> 
<strong>Task 2:</strong> Complete the following stub code to ensure the test passes for two observations in the sequence
</div>

In [18]:
def TestTwoObservations(observation_sequence, sequence_probabilities, tolerance):
    # This flag will stay True if your test passes
    test_passes = True

    # Pull out the two observations in the sequence
    obs1 = observation_sequence.seq[0]
    obs2 = observation_sequence.seq[1]

    token_count = 0
    
    # Loop through possible symbols (letter keys) for the first observation
    for symbol_obs1 in symbols:
        # Find the key that generates the symbol
        key1 = get_key_for_symbol(symbol_obs1)

        # TODO compute the prior, likelihood and accumulated probability after the first observation
        # ...

        # Loop through possible symbols (letter keys) for the second observation
        for symbol_obs2 in symbols:
            # Find the key that generates the symbol
            key2 = get_key_for_symbol(symbol_obs2)
            
            # TODO compute the prior, likelihood and accumulated probability after the second observation
            # ...

            # Compare accumulated probability to that reported from our recursive token passing method
            if not (abs(obs_seq2_prob[token_count] - acc_prob) < tol):           
                test_passes = False
            token_count += 1
    return test_passes

Run the test for the case of two observations in a sequence (no code changes required)

In [19]:
# Do not change the tolerance
tol = 0.000001

# A sequence with a two Observations
obs_seq2 = ObservationSequence([Observation(0.1,0.1),Observation(0.15,-0.1)]) #aa + noise
# The probabilities associated with the 16 possible token hypotheses, i.e. 'aa','ab','ac','ad', 'ba', 'bb', ... ,'dd'
obs_seq2_prob = [0.001542,0.000846,0.001087,0.000596,\
                 0.001034,0.000567,0.000728,0.000400,\
                 0.001034,0.000567,0.000728,0.000400,\
                 0.000693,0.000380,0.000488,0.000268,]

if (TestTwoObservations(obs_seq2,obs_seq2_prob,tol)):
    print("PASSED: well done!")
else:
    print("FAILED: try again!")

NameError: name 'acc_prob' is not defined

## Decoding with Substitutions and Deletions

Our first decoder has some limitations. In particular, it cannot identify a hypothesis which contains more symbols than the number of observations. However, we can easily modify the decoder to handle both substitutions and deletions of observations. We do this by modifying our <em>propagate</em> function so that instead of generating $k$ transitions ($k$ tokens), each transition modelling a substituion of an observation with a letter, we generate $k+1$ transitions ($k+1$ tokens). The additional transition (token) models an $\epsilon$-transition (see figure below).

<img src="Rich_Interactions_in_MR/imgs/simple_epsilon_chain.svg">

Incorporating the $\epsilon$-transition, we can see below that the decoder is able to ignore noisy observations. For instance, for the observation sequence modelling letter keys: B, A, (noise), D, the decoder is able to delete the noisy observation. If we had a more sophisticated language model, the decoder would now be able to correct misspellings and accidental touches on letter keys.

<div class="alert alert-info">
<strong>GROUP DISCUSSION:</strong> 
<ul>
    <li>
        Compare the deletion penalty value used here to the probabilities determined for the single observation sequence examined above?
    </li>
    <li>
        Roughly, at what distance does the spurious third observation in the four observation sequence below start to be interpreted as a legitimate touch for the best hypothesis?
    </li>
</ul>
</div>

In [20]:
symbols = ['a','b','c','d','ɛ'] #Epsilon represents a delete observation event (i.e. do not generate a hypothesis)
#obs_seq = ObservationSequence([]) #No observations
#obs_seq = ObservationSequence([Observation(0.1,1.1),Observation(0.15,-0.1),Observation(1.05,0.95)]) #bad + noise
obs_seq = ObservationSequence([Observation(0.1,1.1),Observation(0.15,-0.1),Observation(3.0,-2.7),Observation(1.05,0.95)]) #b - a - (Noisy input far away from any key) - d + noise
completed_tokens = []
deletion_penalty = 0.02

def get_key_for_symbol(symbol):
    for key in keys:
        if key.letter == symbol:
            return key
    return None

def propagate(token, obs_seq, completed_tokens):
    # Which observation is the token in?
    ix = token.obs_index
    # Are there more observations?
    next_ix = ix + 1
    if next_ix == len(obs_seq.seq):
        # No more observations, add token to the list of completed tokens
        completed_tokens.append(token)
    else:
        # There is another observation, propagage a token per symbol to the next observation
        next_obs = obs_seq.seq[next_ix]
        for symbol in symbols:
            if symbol == 'ɛ':
                # Delete observation
                acc_prob = token.acc_prob * deletion_penalty
                # Create a new token for the next observation index but don't accumulate a hypothesis
                new_token = Token(token.hypo, acc_prob, next_ix)
                propagate(new_token, obs_seq, completed_tokens)
            else:
                # Substitute observation for symbol
                key = get_key_for_symbol(symbol)
                prior = get_prior(token.hypo+symbol, lm)
                likelihood = get_likelihood(next_obs.x, next_obs.y, key)
                acc_prob = token.acc_prob * prior * likelihood
                new_token = Token(token.hypo+symbol, acc_prob, next_ix)
                propagate(new_token, obs_seq, completed_tokens)

seed_token = Token("", 1.0, -1)
propagate(seed_token, obs_seq, completed_tokens)
results = sorted(completed_tokens, key=lambda token: token.acc_prob, reverse=True)

# Print out sequence hypotheses and their probabilities
print_sorted_tokens(results)

letter sequence
nth best, hypothesis, probability
1, bad, 0.0000012
2, bcd, 0.0000009
3, dad, 0.0000008
4, bac, 0.0000008
5, bab, 0.0000007
6, bbd, 0.0000007
7, aad, 0.0000007
8, bd, 0.0000006
9, ad, 0.0000006
10, ba, 0.0000006
11, dcd, 0.0000006
12, bcc, 0.0000006
13, dac, 0.0000005
14, bcb, 0.0000005
15, bdd, 0.0000005
...
611, bba, 2.95e-12
612, cbb, 2.93e-12
613, db, 2.85e-12
614, ba, 2.75e-12
615, ccba, 2.75e-12
616, ddba, 2.75e-12
617, cb, 2.73e-12
618, cdbc, 2.61e-12
619, cdbb, 2.36e-12
620, adba, 2.25e-12
621, cbba, 2.14e-12
622, dba, 2.08e-12
623, cdb, 2.07e-12
624, cba, 1.99e-12
625, cdba, 1.51e-12


## Beam Pruning

Our prior decoders will generate exponential growth in the number of tokens as a function of the number of observations. At observation index 1 we have propagated $k$ tokens, at observation index 2 we have propaged $k^2$ tokens, and so on.

To keep the search complexity tractable we observe that many tokens that we are propagated are highly unlikely to contain the most probable hypotheses. These tokens can be filtered out using <em>beam pruning</em>. For every observation index we store the highest accumulated probability we have generated so far for that particular observation. We then impose a condition before we propagate a token: only propagate the token if the token's accumulated probability is within a certain distance to the highest accumulated probability for the next observation. This distance is known as the <em>beam width</em> and it is a parameter that can be used to set the operating point (or trade-off) between accuracy and time when searching for a hypothesis. The narrower the beam the more greedy the search becomes.

We can see that the decoder generates less hypotheses as some hypotheses are eliminated in the search.

<div class="alert alert-info">
<strong>GROUP DISCUSSION:</strong> 
<ul>
    <li>
        Examine how increasing the beam width to around 0.7 affects the resulting hypotheses?
    </li>
    <li>
        What is a potential downside of introducing beam pruning?
    </li>
</ul>
</div>

In [21]:
class ObservationSequence:
    def __init__(self, seq):
        self.seq = seq
        self.beam = [0.0]*len(seq) 

symbols = ['a','b','c','d','ɛ'] #Epsilon represents a delete observation event (i.e. do not generate a hypothesis)
#obs_seq = ObservationSequence([]) #No observations
#obs_seq = ObservationSequence([Observation(0.1,1.1),Observation(0.15,-0.1),Observation(1.05,0.95)]) #bad + noise
obs_seq = ObservationSequence([Observation(0.1,1.1),Observation(0.15,-0.1),Observation(3.0,-2.7),Observation(1.05,0.95)]) #b - a - (Noisy input far away from any key) - d + noise
completed_tokens = []
deletion_penalty = 0.02
beam_width = 0.2

def get_key_for_symbol(symbol):
    for key in keys:
        if key.letter == symbol:
            return key
    return None

def beam_prune(p, obs_seq, obs_seq_index):
    bp = obs_seq.beam[obs_seq_index]
    if p > bp:
        obs_seq.beam[obs_seq_index] = p
        return False
    else:
        if p < beam_width * bp:
            return True
        else:
            return False

def propagate(token, obs_seq, completed_tokens):
    ix = token.obs_index
    next_ix = ix + 1
    if next_ix == len(obs_seq.seq):
        completed_tokens.append(token)
    else:
        next_obs = obs_seq.seq[next_ix]
        for symbol in symbols:
            if symbol == 'ɛ':
                acc_prob = token.acc_prob * deletion_penalty
                new_token = Token(token.hypo, acc_prob, next_ix)
                if not beam_prune(acc_prob, obs_seq, next_ix):
                    propagate(new_token, obs_seq, completed_tokens)
            else:
                key = get_key_for_symbol(symbol)
                prior = get_prior(token.hypo+symbol, lm)
                likelihood = get_likelihood(next_obs.x, next_obs.y, key)
                acc_prob = token.acc_prob * prior * likelihood
                new_token = Token(token.hypo+symbol, acc_prob, next_ix)
                if not beam_prune(acc_prob, obs_seq, next_ix):
                    propagate(new_token, obs_seq, completed_tokens)

seed_token = Token("", 1.0, -1)
propagate(seed_token, obs_seq, completed_tokens)
results = sorted(completed_tokens, key=lambda token: token.acc_prob, reverse=True)

# Print out sequence hypotheses and their probabilities
print_sorted_tokens(results)

letter sequence
nth best, hypothesis, probability
1, bad, 0.0000012
2, bcd, 0.0000009
3, dad, 0.0000008
4, bac, 0.0000008
5, bab, 0.0000007
6, bbd, 0.0000007
7, aad, 0.0000007
8, bd, 0.0000006
9, ad, 0.0000006
10, ba, 0.0000006
11, dcd, 0.0000006
12, bcc, 0.0000006
13, dac, 0.0000005
14, bcb, 0.0000005
15, bdd, 0.0000005
...
69, aca, 0.0000002
70, a, 0.0000002
71, adc, 0.0000002
72, adb, 0.0000001
73, aba, 0.0000001
74, aacd, 4.72e-09
75, aacc, 3.01e-09
76, aacb, 2.73e-09
77, aac, 2.38e-09
78, aaca, 1.74e-09
79, aaad, 3.88e-10
80, aaac, 2.47e-10
81, aaab, 2.24e-10
82, aaa, 1.95e-10
83, aaaa, 1.43e-10


## Decoding with Substitutions, Deletions and Insertions

Our latest decoder can substitute observations into either letters or empty strings ($\epsilon$-transitions). However, it cannot model the fact that a user may have accidentally failed to touch a letter key. To correct this behaviour we need to model <em>insertion</em> transitions. This can be done by adding <em>self-loops</em> to each state, see the figure below. Each self-loop can generate any symbol.

To model such insertions we modify our <em>propagate</em> function again. Instead of simply propagating $k+1$ tokens to observation index $i+1$, we also propagate $k$ tokens to observation index $i$, the observation index of the token under consideration.

Our search space is no longer increasing exponentially: it is infinite. The reason is that the decoding process can stay at observation index $i$ indefinately and keep generating new tokens. Therefore insertions need to carry an <em>insertion penalty</em>. If the beam width and the insertion penalty are set correctly, the search becomes tractable again.

<img src="Rich_Interactions_in_MR/imgs/simple_self-loop_chain.svg">

<div class="alert alert-info">
<strong>GROUP DISCUSSION:</strong> 
<ul>
    <li>
        Examine how adjusting the insertion penalty affects the length of the resulting hypotheses (advise not going above ~3)?
    </li>
    <li>
        How does the concept of an insertion penalty loosely relate to the way we have modelled the likelihood of an observation?
    </li>
</ul>
</div>

In [22]:
class ObservationSequence:
    def __init__(self, seq):
        self.seq = seq
        self.beam = [0.0]*len(seq) 

symbols = ['a','b','c','d','ɛ'] #Epsilon represents a delete observation event (i.e. do not generate a hypothesis)
#obs_seq = ObservationSequence([]) #No observations
#obs_seq = ObservationSequence([Observation(0.1,1.1),Observation(0.15,-0.1),Observation(1.05,0.95)]) #bad + noise
obs_seq = ObservationSequence([Observation(0.1,1.1),Observation(0.15,-0.1),Observation(3.0,-2.7),Observation(1.05,0.95)]) #b - a - (Noisy input far away from any key) - d + noise
completed_tokens = []
deletion_penalty = 0.02
insertion_penalty = 1.2
beam_width = 0.2

def get_key_for_symbol(symbol):
    for key in keys:
        if key.letter == symbol:
            return key
    return None

def beam_prune(p, obs_seq, obs_seq_index):
    bp = obs_seq.beam[obs_seq_index]
    if p > bp:
        obs_seq.beam[obs_seq_index] = p
        return False
    else:
        if p < beam_width * bp:
            return True
        else:
            return False

def propagate(token, obs_seq, completed_tokens):
    # Which observation is the token in?
    ix = token.obs_index
    # Are there more observations?
    next_ix = ix + 1
    if next_ix == len(obs_seq.seq):
        # No more observations, add token to the list of completed tokens
        completed_tokens.append(token)
    else:
        # There is another observation, propagage tokens for all symbols to the next observation index
        next_obs = obs_seq.seq[next_ix]
        for symbol in symbols:
            if symbol == 'ɛ':
                acc_prob = token.acc_prob * deletion_penalty
                new_token = Token(token.hypo, acc_prob, next_ix)
                if not beam_prune(acc_prob, obs_seq, next_ix):
                    propagate(new_token, obs_seq, completed_tokens)
            else:
                key = get_key_for_symbol(symbol)
                prior = get_prior(token.hypo+symbol, lm)
                likelihood = get_likelihood(next_obs.x, next_obs.y, key)
                acc_prob = token.acc_prob * prior * likelihood
                new_token = Token(token.hypo+symbol, acc_prob, next_ix)
                if not beam_prune(acc_prob, obs_seq, next_ix):
                    propagate(new_token, obs_seq, completed_tokens)
    # Propagate tokens for all symbols (except epsilon) within the same observation index
    if ix >= 0:
        for symbol in symbols:
            if not symbol == 'ɛ':
                key = get_key_for_symbol(symbol)
                prior = get_prior(token.hypo+symbol, lm)
                obs = obs_seq.seq[ix]
                acc_prob = token.acc_prob * prior * insertion_penalty
                # Create a new token but don't increment the observation index
                new_token = Token(token.hypo+symbol, acc_prob, ix)
                if not beam_prune(acc_prob, obs_seq, ix):
                    propagate(new_token, obs_seq, completed_tokens)

seed_token = Token("", 1.0, -1)
propagate(seed_token, obs_seq, completed_tokens)
results = sorted(completed_tokens, key=lambda token: token.acc_prob, reverse=True)

# Print out sequence hypotheses and their probabilities
print_sorted_tokens(results)

letter sequence
nth best, hypothesis, probability
1, bad, 0.0000012
2, bcd, 0.0000009
3, dad, 0.0000008
4, bac, 0.0000008
5, bab, 0.0000007
6, bbd, 0.0000007
7, aad, 0.0000007
8, bd, 0.0000006
9, ad, 0.0000006
10, ba, 0.0000006
11, dcd, 0.0000006
12, bcc, 0.0000006
13, dac, 0.0000005
14, bcb, 0.0000005
15, bdd, 0.0000005
...
213, aaabd, 1.16e-10
214, aaacd, 1.16e-10
215, aaadd, 1.16e-10
216, aaaca, 7.42e-11
217, aaacb, 7.42e-11
218, aaacc, 7.42e-11
219, aaacd, 7.42e-11
220, aaaba, 6.71e-11
221, aaabb, 6.71e-11
222, aaabc, 6.71e-11
223, aaabd, 6.71e-11
224, aaaaa, 4.28e-11
225, aaaab, 4.28e-11
226, aaaac, 4.28e-11
227, aaaad, 4.28e-11


We can see from the output that our new decoder is capable of generating longer hypotheses than there are observations. However, the most likely hypothesis remains the same as before. The reason is our parameter settings and our language model. Since our language model simply distributes the probability mass evenly over our four possible letters and does not model probabilities over letter sequences, there is no structure that allows the decoder to sophisticatedly decide to insert a letter. For insertions to be used effectively, a high-quality language model is required.

## Log probabilities

The previous code works but will not scale very well due to numerical underflow. To avoid this, statistical decoding is usually working with probabilities on a logarithmic scale (_log-probs_).

To transform a probability $P$ to the logarithmic scale we compute the natural logarithm, $\ln(P)$. To transform a log-prob back into a regular probability we compute $\exp(\ln(P))$. Adding log-probs has the same effect as multiplying regular probabilties.

An example: 

Let $P_1 = P_2 = 0.5$. 

Then $\ln(P_1) = \ln(P_2) \approx -0.693$. 

$\ln(P_1) + \ln(P_2) \approx -1.386$ and $\exp\left[\ln(P_1) + \ln(P_2)\right] = 0.25$.

In [ ]:
P1 = 0.5
P2 = 0.5

joint_P1_P2 = P1 * P2

# Take log
log_P1 = math.log(P1)
log_P2 = math.log(P2)

# Sum log probs
log_joint_P1_P2 = log_P1 + log_P2

# Take exp(log(P))
e_log_joint_P1_P2 = math.exp(log_joint_P1_P2)

print('Multiplied Probabilities - Summed Log Probabilities')
print('%.3g - %.3g' % (joint_P1_P2,e_log_joint_P1_P2))

<p>
<div class="alert alert-block alert-success"> 
<h3>Exercise 2: Use Log Probabilities</h3> 
<br>
Work in your group and extend the decoder to use log probabilities throughout. Check that the results returned are the same as those above.
</div>
</p>

In [23]:
class ObservationSequence:
    def __init__(self, seq):
        self.seq = seq
        self.beam = [0.0]*len(seq) 

symbols = ['a','b','c','d','ɛ'] #Epsilon represents a delete observation event (i.e. do not generate a hypothesis)
#obs_seq = ObservationSequence([]) #No observations
#obs_seq = ObservationSequence([Observation(0.1,1.1),Observation(0.15,-0.1),Observation(1.05,0.95)]) #bad + noise
obs_seq = ObservationSequence([Observation(0.1,1.1),Observation(0.15,-0.1),Observation(3.0,-2.7),Observation(1.05,0.95)]) #b - a - (Noisy input far away from any key) - d + noise
completed_tokens = []
deletion_penalty = 0.02
insertion_penalty = 1.2
beam_width = 0.2

def get_key_for_symbol(symbol):
    for key in keys:
        if key.letter == symbol:
            return key
    return None

def beam_prune(p, obs_seq, obs_seq_index):
    bp = obs_seq.beam[obs_seq_index]
    if p > bp:
        obs_seq.beam[obs_seq_index] = p
        return False
    else:
        if p < beam_width * bp:
            return True
        else:
            return False

def propagate(token, obs_seq, completed_tokens):
    # Which observation is the token in?
    ix = token.obs_index
    # Are there more observations?
    next_ix = ix + 1
    if next_ix == len(obs_seq.seq):
        # No more observations, add token to the list of completed tokens
        completed_tokens.append(token)
    else:
        # There is another observation, propagage tokens for all symbols to the next observation index
        next_obs = obs_seq.seq[next_ix]
        for symbol in symbols:
            if symbol == 'ɛ':
                acc_prob = token.acc_prob * deletion_penalty
                new_token = Token(token.hypo, acc_prob, next_ix)
                if not beam_prune(acc_prob, obs_seq, next_ix):
                    propagate(new_token, obs_seq, completed_tokens)
            else:
                key = get_key_for_symbol(symbol)
                prior = get_prior(token.hypo+symbol, lm)
                likelihood = get_likelihood(next_obs.x, next_obs.y, key)
                acc_prob = token.acc_prob * prior * likelihood
                new_token = Token(token.hypo+symbol, acc_prob, next_ix)
                if not beam_prune(acc_prob, obs_seq, next_ix):
                    propagate(new_token, obs_seq, completed_tokens)
    # Propagate tokens for all symbols (except epsilon) within the same observation index
    if ix >= 0:
        for symbol in symbols:
            if not symbol == 'ɛ':
                key = get_key_for_symbol(symbol)
                prior = get_prior(token.hypo+symbol, lm)
                obs = obs_seq.seq[ix]
                acc_prob = token.acc_prob * prior * insertion_penalty
                # Create a new token but don't increment the observation index
                new_token = Token(token.hypo+symbol, acc_prob, ix)
                if not beam_prune(acc_prob, obs_seq, ix):
                    propagate(new_token, obs_seq, completed_tokens)

seed_token = Token("", 1.0, -1)
propagate(seed_token, obs_seq, completed_tokens)
results = sorted(completed_tokens, key=lambda token: token.acc_prob, reverse=True)

# Print out sequence hypotheses and their probabilities
print_sorted_tokens(results)

letter sequence
nth best, hypothesis, probability
1, bad, 0.0000012
2, bcd, 0.0000009
3, dad, 0.0000008
4, bac, 0.0000008
5, bab, 0.0000007
6, bbd, 0.0000007
7, aad, 0.0000007
8, bd, 0.0000006
9, ad, 0.0000006
10, ba, 0.0000006
11, dcd, 0.0000006
12, bcc, 0.0000006
13, dac, 0.0000005
14, bcb, 0.0000005
15, bdd, 0.0000005
...
213, aaabd, 1.16e-10
214, aaacd, 1.16e-10
215, aaadd, 1.16e-10
216, aaaca, 7.42e-11
217, aaacb, 7.42e-11
218, aaacc, 7.42e-11
219, aaacd, 7.42e-11
220, aaaba, 6.71e-11
221, aaabb, 6.71e-11
222, aaabc, 6.71e-11
223, aaabd, 6.71e-11
224, aaaaa, 4.28e-11
225, aaaab, 4.28e-11
226, aaaac, 4.28e-11
227, aaaad, 4.28e-11


# Extension Topic - Statistical Language Modelling

A _statistical language model_ is a model that assigns probabilities to sequences of letters or words.

### Information, Entropy and Perplexity

To understand how to model languages we need to understand what a _message_ is and how we can quantify the information in a message. A message $m$ in some message space $M$ is sent from a _sender_ to a _receiver_. Importantly, for a message $m$ to carry information $I(m)$ from the sender to the recevier the information must not already be known to the receiver. If the information is already known $I(m)=0$. $I(m)$ measures the amount of new information, or _surprisal_, there is in receiving a message. The more surprising the message is to the receiver, the more information content in the message.

$I(m)$ is called _self-information_ and is defined as:

\begin{equation}
I(m) = \log_2\left(\frac{1}{P(m)}\right)=-\log_2\left(P\left(m\right)\right),
\end{equation}

where $P(m)$ is the probability of the message in the message space $M$.

The above equation is better understood by considering a message as communicating an event (such as the outcome of flipping a fair coin: heads or tails). Let information about one event $m_1$ be $I(m_1)$ and another event $m_2$ be $I(m_2)$.

Then, _assuming these events are independent_, the information about _both_ these events would be $I_{both} = I(m_1\cap m_2)= I(m_1) + I(m_2)$.

If we now think about the _probabilities_ of these events we have $P(m_1)$ and $P(m_2)$ and $P_{both} = P(m_1\cap m_2)= P(m_1) \cdot P(m_2)$.

In other words, self-information of independent events _adds_ while the probabilities of the events _multiply_. This is exactly how log-probs operate, which explains the logarithm in the above equation. Since logarithms scale with a constant factor we can set this constant factor to be negative in order to ensure information is a positive quantity. The base of the logarithm determines the unit. By convention the base is usually 2 ($\log_2)$ and thus the unit is in _bits_.

<h4>Example 1:</h4>

Consider a Bernoulli trial involving flipping a fair coin. The probability of either outcome is $0.5$. To communicate the specific outcome of an individual event to the sender is the equivalent of sending a message $m$ with $I(m) = \log_2\left(\frac{1}{0.5}\right) =  \log_2(2) = 1$ bit.

<h4>Example 2:</h4>

Consider we wish to communicate the outcome of having flipped a fair coin four times as either $m_a$ (four heads in a row) or $\bar{m_a}$ (_not_ observing four heads in a row). The probability $P(m_a)$ of observing four heads in a row is $\frac{1}{16}$ and $I(m_a) = -\log_2\left(P\left(m_a\right)\right) = -\log_2\left(\frac{1}{16}\right) = 4$ bits. Communicating the other possible outcome, $\bar{m_a}$, we first find the probability $P(\bar{m_a}) = 1 - P(m_a) = 1 - \frac{1}{16} = \frac{15}{16}$. Then $I(\bar{m_a}) = -\log_2\left(\frac{15}{16}\right) \approx 0.093$ bits.

Entropy $H$ is a measure of the _uncertainty_ in the message space $M$.

\begin{equation}
H(M) = \sum_{m \in M} P(m)I(m)=-\sum_{m \in M}P(m)\log_2\left(P(m)\right)
\end{equation}

Entropy is the _average_ self-information of all messages in the message space. Entropy measures disorder. An entropy of zero means there is no disorder and everything is completely predictable.

Entropy tells us the average bits we need to communicate a message, assuming we have found an optimal coding scheme. The difference between the average bits actually used for communication and the optimal bits truly necessary to encode all messages is a measure of _redundancy_. Natural langauges have high redundancy.

A measure related to entropy is _perplexity_, which measures how well a probability model is at prediction. Perplexity is defined as $2^H$, where $H$ is the entropy of the model.

<h4>Example from Jurafsky and Martin 2000; _Speech and Language Processing_:</h4>

We want to bet on eight horses via a middleman. How do we communicate the horse we want to bet on?. One way is to encode this message using binary representation for each horse, horse 1 becomes 001, horse 2 becomes 010, etc. If we spend the whole day betting and each horse is coded with 3 bits then on average we are sending 3 bits per race.

Can we do better? Assume each horse is not equally likely but instead distributed like this: horse 1: $\frac{1}{2}$; horse 2: $\frac{1}{4}$; horse 3: $\frac{1}{8}$; horse 4: $\frac{1}{16}$; and horses 5--8: $\frac{1}{64}$. Then the entropy of the random variable $X$ that ranges over horses gives us a lower bound on the number of bits: $H(X) = -\sum^8_{i=1}P(i)\log_2\left(P(i)\right)=2$ bits. Such a code can be built by using shorter encoding for more probable horses and longer encodings for less probable horses (for example, the most likely horse can be encoded as 0 while the least likely horse can be coded as 111111. If the horses have equal probability then if we repeat the above calculation we will find that the entropy would be 3.

Perplexity is the weighted average number of choices a random variable has to make. Choosing between 8 equally likely horses ($H = 3$ bits) has a perplexity of $2^3 = 8$. Choosing between the biased horses above ($H = 2$ bits) has a perplexity of $2^2 = 4$.






### Modelling language

Since natural languages are highly redundant it follows that the vast majority of letter and word combinations are highly improbable. We can capture probable sequences of letters and words by counting their occurance in some large representative dataset, a _corpus_. A _language model_ assigns probabilities to sequences of letters and words.

We will demonstrate these ideas by using an example from: Shannon, C.E. 1948. A mathematical theory of communication. _Bell System Technical Journal_ 27: 379–423, 623–656.

Consider a discrete source with five letters \{A,B,C,D,E\}. Chosen with probability of 0.2 (uniformaly sampled) and each choice independent of previous choices we might expect a typical outcome such as: B D C B C E C C C A D C B D D A A E C E E A A B B D A E E C A C E E B A E E C B C E A D

We can build a generative model that generates increasingly English-like language output, as follows:

<b>Zero-order approximation (uniform random sampling from {A-Z}:</b>

<em>XFOML RXKHRJFFJUJ ZLPWCFWKCYJ FFJEYVKCQSGHYD QPAAMKBZAACIBZLHJQD</em>

<b>First-order approximation (sampling according to the probabilities of English letters occurring in common texts):</b>

<em>OCRO HLI RGWR NMIELWIS EU LL NBNESEBYA TH EEI ALHENHTTPA OOBTTVA NAH BRL</em>

<b>Second-order approximation (sampling so that the every letter is dependent on its previous letter according to letter probabilities in English):</b>

<em>ON IE ANTSOUTINYS ARE T INCTORE ST BE S DEAMY ACHIN D ILONASIVE TUCOOWE AT TEASONARE FUSO TIZIN ANDY TOBE SEACE CTISBE</em>

<b>Third-order approximation (sampling similar to second-order approximation, except each letter is now dependent on two previous letters):</b>

<em>IN NO IST LAT WHEY CRATICT FROURE BIRS GROCID PONDENOME OF DEMONSTURES OF THE REPTAGIN IS REGOACTIONA OF CRE</em>

<h4>Zipf's law</h4>

Another striking example of language redudandancy is Zipf's law, which estimates the probability $P_r$ of occurrence of a word in a corpus to be $P_r \propto \frac{1}{r^\alpha}$, where $r$ is the statistical rank of the word in decreasing order and $\alpha$ is close to unity. As an example of this relationship, the 100 most frequently used words in the British National Corpus comprise close to 46% of the entire corpus.

<h4><em>n</em>-gram models</h4>

A _unigram model_ assumes any word will follow any other word at an equal probability. The simplest unigram model possible believes every word is equally probable. So if a language consists of $1000$ words then the probability of occurrence of a word is exactly $\frac{1}{1000}$.

The word "the" occurs 69,971 times in the Brown corpus while the word "rabbit" only occurs 11 times. However, what is the most reasonable choice of word to complete the following sentence fragment: "Just then, the white...". Clearly a unigram model makes too simplistic assumptions about natural languages.

We can handle the case with the white rabbit by probabilistically modelling every single possible word combination that can occur in the world. However, it is clearly intractable to model such a large space, which is countably infinite. Therefore, language models make a _Markov assumption_: the probability of a word only depends on a finite set of previous words.

A bigram model approximates the probability of a word given previous words by the conditional probability of a single preceding word. So instead of computing this probability: $P(\text{rabbit}|\text{Just the other day I saw a})$, a bigram model computes the following approximate probability: $P(\text{rabbit}|\text{a})$.

A trigram model is a minor extension to a bigram model: it has two words of prior context. In general, an $n$-gram model has $n-1$ words of context an is said to be of order $n-1$.

The basic way of training a language model is by counting instances of $n$-grams in corpora (frequency estimation). These counts have to be normalised in order to create a probabilistic model. Fundamentally, one takes the count of say a particular bigram and divide it by the sum of all the bigrams that share the first word (which is identical to the unigram count of the first word). This is a form of _parameter estimation_. In general, we estimate an $n$-gram probability by dividing the observed frequency of a particular sequence by the observed frequency of a prefix. Using such relative frequency to estimate probabilities is an example of maximum likelihood estimation (the likelihood of the training set given the model is maximised). Typically language models use log-probs.

The typical way to evaluate a language model is to train it on a training set and test it on a separate test set. The lower the perplexity the better the language model is in predicting text on the test set.

### Smoothing

Our previous language modelling cannot account for $n$-grams we have not previously observed in our training data. Fundamentally, the problem is that our language model is overestimating the probabilities of observed sequences and underestimting unobserved sequences by assigning them zero probability mass. The solution is to distribute some of the probability mass in the language model to unobserved sequences.

The simplest method is to add one to every count of every $n$-gram in the model (this is known as the Laplace method):

\begin{equation}
c^*=c+1,
\end{equation}

where $c$ is the original count and $c^*$ is the modified count. It does solve the problem of zero counts but in practice it results in poor language modelling performance.

A slightly more sophisticated version is known as Good-Turing discounting and it is defined as:

\begin{equation}
c^*=(c+1)\frac{N_{c+1}}{N_c},
\end{equation}

where $c$ is the original count and $c^*$ is the modified count as before and $N_c$ and $N_{c+1}$ are the number of $n$-grams that occur exactly $c$ and $c+1$ times. An obvious problem is when $N_c = 0$. One solution is to plot $\log(c)$ against $\log(Z_c)$, where $Z_c$ is:

\begin{equation}
Z_c = \frac{2N_c}{c''-c'},
\end{equation}

where $c''$ and $c'$ are the nearest higher and lower sample counts such that $N_{c''}$ and $N_{c'}$ are positive. A linear fit between $\log(c)$ against $\log(Z_c)$ can then be obtained by ordinary least squares (OLS) regression and used to infer $N_c$ that are unobservable.

See <a href="https://www.grsampson.net/AGtf.html">Good–Turing Frequency Estimation Without Tears</a> for a complete explanation.

<p>
<div class="alert alert-block alert-success"> 
<h3>Exercise 3: Improve the Statistical Decoder</h3> 
<br>
The code for the statistical decoder you were provided with has a few limitations. In this exercise you will work in your group and introduce a better language model.  Change the uniform unigram model to a character-level language model that learns probabilities for sequences of characters based on a training text and uses smoothing to estimate the probabilities for unseen character sequences.
</div>
</p>

<div class="alert alert-block alert-success"> 
    <strong>Task 1:</strong> Construct a character-level bigram language model based on the provided training text. It is useful to represent the start of a word as a psuedo character.  
</div>

In [24]:
# Load the training text
f_training_text = open('Rich_Interactions_in_MR/logs/training_text.txt', 'r')
training_text = f_training_text .read()
f_training_text.close()

# Initalize a structure to count character-to-character transitions, include a start-of-word one-way transition denoted '_'
transition_counts = np.matlib.zeros((5,4))
from_ids = {'_':0,'a':1,'b':2,'c':3,'d':4}
to_ids = {'a':0,'b':1,'c':2,'d':3}

# Split the training text into words
training_words = training_text.split(' ')

# Count character transitions
for word in training_words:
    prev_character = '_'
    for character in word:
        # TODO: Update the transition_counts matrix, make use of from_ids and to_ids
        from_id = from_ids[prev_character] # REMOVE
        to_id = to_ids[character] # REMOVE        
        transition_counts[from_id,to_id] += 1 # REMOVE
        
        # Update previous character
        prev_character = character

# TODO: Apply smoothing (if necessary) and convert the transition counts into a useful bigram
smoothed_transition_counts = transition_counts + np.matlib.ones((5,4)) # REMOVE
print(smoothed_transition_counts) # REMOVE
transition_probs = smoothed_transition_counts/smoothed_transition_counts.sum(axis=1) # REMOVE
print(transition_probs) # REMOVE

[[1.624e+03 1.083e+03 1.204e+03 1.093e+03]
 [3.040e+02 1.368e+03 5.950e+02 1.201e+03]
 [1.527e+03 3.180e+02 1.730e+02 1.000e+00]
 [7.550e+02 1.000e+00 1.470e+02 2.950e+02]
 [1.093e+03 1.000e+00 2.840e+02 1.410e+02]]
[[3.24540368e-01 2.16426859e-01 2.40607514e-01 2.18425260e-01]
 [8.76585928e-02 3.94463668e-01 1.71568627e-01 3.46309112e-01]
 [7.56315007e-01 1.57503715e-01 8.56859832e-02 4.95294700e-04]
 [6.30217028e-01 8.34724541e-04 1.22704508e-01 2.46243740e-01]
 [7.19552337e-01 6.58327847e-04 1.86965109e-01 9.28242265e-02]]


<div class="alert alert-block alert-success"> 
    <strong>Task 2:</strong> Modify the get_prior method to use use the computed bigram.  
</div>

In [25]:
# REMOVE
lm = {'_a':3.24540368e-01,'_b':2.16426859e-01,'_c':2.40607514e-01,'_d':2.18425260e-01,\
'aa':8.76585928e-02,'ab':3.94463668e-01,'ac':1.71568627e-01,'ad':3.46309112e-01,\
'ba':7.56315007e-01,'bb':1.57503715e-01,'bc':8.56859832e-02,'bd':4.95294700e-04,\
'ca':6.30217028e-01,'cb':8.34724541e-04,'cc':1.22704508e-01,'cd':2.46243740e-01,\
'da':7.19552337e-01,'db':6.58327847e-04,'dc':1.86965109e-01,'dd':9.28242265e-02}

# Populate the bigram with computed probabilities
#lm = {'_a':0.25,'_b':0.25,'_c':0.25,'_d':0.25,\
#      'aa':0.25,'ab':0.25,'ac':0.25,'ad':0.25,\
#      'ba':0.25,'bb':0.25,'bc':0.25,'bd':0.25,\
#      'ca':0.25,'cb':0.25,'cc':0.25,'cd':0.25,\
#      'da':0.25,'db':0.25,'dc':0.25,'dd':0.25}

# Modify get_prior to return prior for specified context
#def get_prior(context, lm):
#    c = context[-1]
#    return lm[c]

# REMOVE
def get_prior(context, lm):
    if (len(context) == 1):
        context = '_' + context
    c = context[-2:]
    return lm[c]

<div class="alert alert-block alert-success"> 
    <strong>Task 3:</strong> Examine the effect of the new language model on decoder performance.
</div>

In [26]:
#obs_seq = ObservationSequence([]) #No observations
#obs_seq = ObservationSequence([Observation(0.1,1.1)])
obs_seq = ObservationSequence([Observation(0.1,1.1),Observation(0.15,-0.1),Observation(1.05,0.95)]) #bad + noise
#obs_seq = ObservationSequence([Observation(0.1,1.1),Observation(0.15,-0.1),Observation(3.0,-2.7),Observation(1.05,0.95)]) #b - a - (Noisy input far away from any key) - d + noise
completed_tokens = []
deletion_penalty = 0.02
insertion_penalty = 1.2
beam_width = 0.2

def propagate(token, obs_seq, completed_tokens):
    # Which observation is the token in?
    ix = token.obs_index
    # Are there more observations?
    next_ix = ix + 1
    if next_ix == len(obs_seq.seq):
        # No more observations, add token to the list of completed tokens
        completed_tokens.append(token)
    else:
        # There is another observation, propagate tokens for all symbols to the next observation index
        next_obs = obs_seq.seq[next_ix]
        for symbol in symbols:
            if symbol == 'ɛ':
                acc_prob = token.acc_prob * deletion_penalty
                new_token = Token(token.hypo, acc_prob, next_ix)
                if not beam_prune(acc_prob, obs_seq, next_ix):
                    propagate(new_token, obs_seq, completed_tokens)
            else:
                key = get_key_for_symbol(symbol)
                prior = get_prior(token.hypo+symbol, lm)
                likelihood = get_likelihood(next_obs.x, next_obs.y, key)
                acc_prob = token.acc_prob * prior * likelihood
                new_token = Token(token.hypo+symbol, acc_prob, next_ix)
                if not beam_prune(acc_prob, obs_seq, next_ix):
                    propagate(new_token, obs_seq, completed_tokens)
    # Propagate tokens for all symbols (except epsilon) within the same observation index
    if ix >= 0:
        for symbol in symbols:
            if not symbol == 'ɛ':
                key = get_key_for_symbol(symbol)
                prior = get_prior(token.hypo+symbol, lm)
                obs = obs_seq.seq[ix]
                acc_prob = token.acc_prob * prior * insertion_penalty
                # Create a new token but don't increment the observation index
                new_token = Token(token.hypo+symbol, acc_prob, ix)
                if not beam_prune(acc_prob, obs_seq, ix):
                    propagate(new_token, obs_seq, completed_tokens)

seed_token = Token("", 1.0, -1)
propagate(seed_token, obs_seq, completed_tokens)
results = sorted(completed_tokens, key=lambda token: token.acc_prob, reverse=True)

# Print out sequence hypotheses and their probabilities
print_sorted_tokens(results)

letter sequence
nth best, hypothesis, probability
1, bad, 0.0002221
2, bada, 0.0001917
3, bab, 0.0001459
4, dad, 0.0001429
5, baba, 0.0001324
6, dada, 0.0001234
7, babad, 0.0000954
8, babad, 0.0000954
9, dab, 0.0000939
10, badab, 0.0000908
11, abad, 0.0000865
12, daba, 0.0000852
13, baba, 0.0000844
14, badaba, 0.0000824
15, babada, 0.0000824
...
217, aababa, 0.0000054
218, aababa, 0.0000054
219, aabad, 0.0000052
220, aaca, 0.0000051
221, aadab, 0.0000050
222, aadc, 0.0000048
223, aabada, 0.0000045
224, aadaba, 0.0000045
225, aaa, 0.0000020
226, aaab, 0.0000009
227, aaaba, 0.0000008
228, aaad, 0.0000008
229, aaada, 0.0000007
230, aaac, 0.0000004
231, aaabab, 0.0000004
